In [4]:
import sys
import os
from pprint import pprint as pp
from glob import glob
sys.path.append('../')

from utils import ConfigHandler
from index import IndexHandler, BabelHash
from query_match import QueryMatch, QueryMatchHandler

config = ConfigHandler()
indexHandler = IndexHandler()
qm_handler = QueryMatchHandler()

#print(os.path.abspath(__file__))

#Testando o upload com um dos partial indexes
#filename=glob(f'{config.value_index_filename}.part*')[1]
#print(f'filename {filename}')


indexHandler.load_indexes(config.value_index_filename, config.schema_index_filename)

print('Schema Index:\n')
pp(indexHandler.schema_index)

from utils import ConfigHandler, Similarity
from keyword_match import KeywordMatchHandler
similarity = Similarity(indexHandler.schema_index)

query = ["papers", "making", "database", "systems", "usable"]
#query = ["authors", "papers", "conference", "vldb", "2002", "1995"]
#query = ["making", "database", "systems","usable"]
#query = ["author", "university of michigan", "area", "databases", "papers", "citations", "5000"]
#query = ["apple","stores"]
#print(f'Testing SKMGen for query {query}')

kwHandler = KeywordMatchHandler(similarity)
skm_matches = kwHandler.schema_keyword_match_generator(query, indexHandler.schema_index)
skm_matches = kwHandler.filter_kwmatches_by_segments(skm_matches, segments=["making database systems usable"])

vkm_matches = kwHandler.value_keyword_match_generator(query, indexHandler.value_index)
vkm_matches = kwHandler.filter_kwmatches_by_segments(vkm_matches, segments=["making database systems usable"])
#vkm_matches = kwHandler.filter_kwmatches_by_segments(vkm_matches, ["making database systems usable"])
#print(len(vkm_matches) + len(skm_matches))
#qm_handler.generate_query_matches(query, skm_matches | vkm_matches, segments=["making database systems usable"])


Schema Index:

{'usable': 'usable making database systems', 'making': 'usable making database systems', 'database': 'usable making database systems', 'systems': 'usable making database systems'}
JOURNAL.s(*{papers})
DOMAIN.s(name{making})
CONFERENCE.s(name{making})
JOURNAL.s(name{making})
PUBLICATION.s(abstract{making})
ORGANIZATION.s(*{systems})
AUTHOR.s(name{making})
PUBLICATION.s(*{papers})
PUBLICATION.s(citation_num{papers})
PUBLICATION.s(title{papers})
PUBLICATION.s(reference_num{papers})
ORGANIZATION.s(name{making})
VKM:  papers
VKM:  paper
VKM:  paper
VKM:  making
VKM:  make
VKM:  making
VKM:  database
VKM:  database
VKM:  database
VKM:  systems
VKM:  systems
VKM:  system
VKM:  usable
VKM:  usable
VKM:  usable
{'usable': 'usable making database systems', 'making': 'usable making database systems', 'database': 'usable making database systems', 'systems': 'usable making database systems'}
JOURNAL.v(name{papers})
PUBLICATION.v(title{papers})
KEYWORD.v(keyword{papers})
AUTHOR.v(name

In [5]:
skm_matches

{JOURNAL.s(*{papers}),
 PUBLICATION.s(*{papers}),
 PUBLICATION.s(citation_num{papers}),
 PUBLICATION.s(reference_num{papers}),
 PUBLICATION.s(title{papers})}

In [12]:
sorted([(k, len(v)) for k,v in raw_vkm.items()], key=lambda x:x[1], reverse=True)

[(PUBLICATION.v(title{databases}), 14248),
 (PUBLICATION.v(title{area}), 12233),
 (PUBLICATION.v(title{author}), 7194),
 (PUBLICATION.v(title{papers}), 4034),
 (PUBLICATION.v(title{citations}), 317),
 (KEYWORD.v(keyword{area}), 99),
 (PUBLICATION.v(title{author,papers}), 97),
 (PUBLICATION.v(title{5000}), 66),
 (PUBLICATION.v(title{area,papers}), 37),
 (PUBLICATION.v(title{area,author}), 36),
 (PUBLICATION.v(title{databases,papers}), 29),
 (PUBLICATION.v(title{citations,author}), 27),
 (PUBLICATION.v(title{databases,area}), 19),
 (PUBLICATION.v(title{databases,author}), 14),
 (PUBLICATION.v(title{citations,papers}), 13),
 (KEYWORD.v(keyword{databases}), 10),
 (KEYWORD.v(keyword{author}), 9),
 (AUTHOR.v(name{area}), 9),
 (CONFERENCE.v(name{databases}), 9),
 (ORGANIZATION.v(name{area}), 6),
 (AUTHOR.v(name{author}), 2),
 (CONFERENCE.v(name{area}), 2),
 (PUBLICATION.v(title{citations,area}), 2),
 (PUBLICATION.v(title{citations,databases}), 2),
 (JOURNAL.v(name{area}), 1),
 (DOMAIN.v(name{

In [6]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



In [ ]:
def generate_query_matches(self, keyword_query, keyword_matches, **kwargs):
        #Input:  A keyword query Q, The set of non-empty non-free tuple-sets Rq
        #Output: The set Mq of query matches for Q
        max_qm_size = kwargs.get('max_qm_size',5)
        segments = kwargs.get('segments', [])
        self.query_matches = []
        set_query_match = set()
        
        # len(segments)+1
        # min(len(segments), max_qm_size)+1
        for i in range(1, len(segments)+1):
            print("combining ",i)
            count=0
            for candidate_match in itertools.combinations(keyword_matches,i):
                count+=1
                #print("candidate query match: {}".format(candidate_match))
                if self.has_minimal_cover(candidate_match,keyword_query):
                #and \
                #(len(sements) != 0  and self.check_segments(segments, candidate_match):
                    merged_query_match = self.merge_schema_filters(candidate_match)

                    query_match = QueryMatch(merged_query_match)

                    #TODO: checking user group
                    self.query_matches.append(query_match)
                    set_query_match.add(query_match)
            #print(len(self.query_matches), count)

In [ ]:
from math import comb

sum([comb(len(vkm_matches) + len(skm_matches), i) for i in range(len(query) + 1)])

In [ ]:
qm_handler.rank_query_matches(indexHandler.value_index, indexHandler.schema_index, similarity, log_score=False)
for i, query_match in enumerate(qm_handler.query_matches):
    print(i, query_match, query_match.total_score)

In [ ]:
skm_matches

In [ ]:
score_list  = []
a = (vkm_matches)

In [ ]:
for k in query:
    tuple_index = indexHandler.get_value_mappings(k)
    print("for {}".format(k))
    print('\n'.join(['{}.{}: {}'.format(x,y, len(tuple_index[1][x][y])) for x in tuple_index[1] for y in tuple_index[1][x]]))
    print("-"*30)

In [ ]:
database_index

In [ ]:
making_index = indexHandler.get_value_mappings("making")

In [ ]:
making_index

In [ ]:
system_index = indexHandler.get_value_mappings("system")

In [ ]:
usable_index = indexHandler.get_value_mappings("usable")

In [ ]:
set(making_index[1]['author']['name']) & set(system_index[1]['author']['name']) & set(usable_index[1]['author']['name'])

In [ ]:
import sys
sys.path.append('../')

from utils import ConfigHandler, Similarity
from index import IndexHandler
from keyword_match import KeywordMatchHandler
from query_match import QueryMatch, QueryMatchHandler


config = ConfigHandler()
indexHandler = IndexHandler()
indexHandler.load_indexes(config.value_index_filename, config.schema_index_filename)
similarity = Similarity(indexHandler.schema_index)

#query = ["papers", "making", "database", "systems", "usable"]
#query = ["authors", "papers", "conference", "vldb", "2002", "1995"]
query = ["university", "michigan"]
#print(f'Testing SKMGen for query {query}')
kwHandler = KeywordMatchHandler(similarity)
skm_matches = kwHandler.schema_keyword_match_generator(query, indexHandler.schema_index)
#print (skm_matches)

print(f'Testing VKMGen for query {query}')
vkm_matches = kwHandler.value_keyword_match_generator(query, indexHandler.value_index)
print (vkm_matches | skm_matches)


In [ ]:
query = ["2000"]
#print(f'Testing SKMGen for query {query}')
kwHandler = KeywordMatchHandler(similarity)
skm_matches = kwHandler.schema_keyword_match_generator(query, indexHandler.schema_index)


In [ ]:
lst_01 = list(vkm_matches)

In [ ]:
d_set = [QueryMatch([lst_01[7]] + [lst_01[8]]), QueryMatch([lst_01[16]])]

In [ ]:
qm_handler = QueryMatchHandler()
qm_handler.query_matches = d_set
qm_handler.rank_query_matches(indexHandler.value_index, indexHandler.schema_index, similarity, log_score=False)

for query_match in qm_handler.query_matches:
    print(query_match, query_match.total_score)

In [ ]:
0.00018171986472526662, 0.00018110873101385524

In [ ]:
[(i,item) for i, item in enumerate(lst_01) if item.table == 'organization']

In [ ]:
vkm_matches